In [1]:
import pandas as pd
#read the file
df = pd.read_csv("NVDA.csv")
#generate the rsi column
df['Price Change'] = df['Close'].diff()
df['Gain'] = df['Price Change'].apply(lambda x: x if x > 0 else 0)
df['Loss'] = df['Price Change'].apply(lambda x: -x if x < 0 else 0)
lookback_period = 14
df['Average Gain'] = df['Gain'].rolling(window=lookback_period, min_periods=14).mean()
df['Average Loss'] = df['Loss'].rolling(window=lookback_period, min_periods=14).mean()
df['RS'] = df['Average Gain'] / df['Average Loss']
df['RSI'] = 100 - (100 / (1 + df['RS']))
df['Date'] = pd.to_datetime(df['Date'])
df=df.dropna()

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


features = df[['RSI', 'Close']].values
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
sequence_length = 30
X, y = [], []
for i in range(len(features_scaled) - sequence_length):
    X.append(features_scaled[i:i+sequence_length-1])  # Exclude the last element
    y.append(features_scaled[i+sequence_length])  

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(64, activation='relu'))
model.add(Dense(2))  

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=150, batch_size=64)

# Evaluate the model on the test data
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)
y_test_data = y_test[:, 1]
y_pred_data = y_pred[:, 1]
rmse = np.sqrt(mean_squared_error(y_test_data, y_pred_data))
print(rmse)

Epoch 1/150
3/3 [==============================] - 4s 36ms/step - loss: 0.4016
Epoch 2/150
3/3 [==============================] - 0s 34ms/step - loss: 0.3365
Epoch 3/150
3/3 [==============================] - 0s 34ms/step - loss: 0.2670
Epoch 4/150
3/3 [==============================] - 0s 32ms/step - loss: 0.1875
Epoch 5/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0960
Epoch 6/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0949
Epoch 7/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0511
Epoch 8/150
3/3 [==============================] - 0s 32ms/step - loss: 0.0494
Epoch 9/150
3/3 [==============================] - 0s 34ms/step - loss: 0.0501
Epoch 10/150
3/3 [==============================] - 0s 34ms/step - loss: 0.0420
Epoch 11/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0382
Epoch 12/150
3/3 [==============================] - 0s 34ms/step - loss: 0.0317
Epoch 13/150
3/3 [==============================]

3/3 [==============================] - 0s 32ms/step - loss: 0.0071
Epoch 104/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0080
Epoch 105/150
3/3 [==============================] - 0s 34ms/step - loss: 0.0064
Epoch 106/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0075
Epoch 107/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0077
Epoch 108/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0068
Epoch 109/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0069
Epoch 110/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0068
Epoch 111/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0063
Epoch 112/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0058
Epoch 113/150
3/3 [==============================] - 0s 33ms/step - loss: 0.0057
Epoch 114/150
3/3 [==============================] - 0s 34ms/step - loss: 0.0056
Epoch 115/150
3/3 [=======================

In [11]:
def predict(future_days, ndays):
    future_days = 10
    predictions = []
    last_n_days = features_scaled[-ndays:].reshape(1, -1, 2)
    for _ in range(future_days):
        next_pred = model.predict(last_n_days)
        last_n_days = np.append(last_n_days[0][1:], next_pred).reshape(1, -1, 2)
        next_pred = next_pred.reshape(1, -1, 2) 
        next_pred = scaler.inverse_transform(next_pred[0])  
        predictions.append(next_pred[0])  
    df_predictions = pd.DataFrame({
        'index': list(range(len(df) + len(predictions), len(df) + len(predictions) + future_days)),
        'Close': [pred[1] for pred in predictions],
    })
    df_predictions.to_csv('predictions.csv', index=False)
    
predict(10, 30)

1/1 [==============================] - 0s 26ms/step
